# Tensorflow Temelleri ve Doğrusal Regresyon
Bu Jupyter Notebook tensorflow'un temel prensiplerini ve basit bir doğrusal regresyon modelini içermektedir. 

In [1]:
import tensorflow as tf # RuntimeWarning çıkarsa sorun değil

/Users/elvan/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
tf.__version__

'1.4.1'